In [1]:
import torch 
import json,pickle,math
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
LSM = pickle.load(open('./davis_ligand_similarity_matrix.pkl', 'rb'))
PSM = pickle.load(open('./davis_protein_similarity_matrix.pkl', 'rb'))
full_df = pd.read_csv(open('./davis_all_pairs.csv','r'))

In [3]:
# df

In [4]:
SMILES = json.load(open('./data/DAVIS/SMILES.txt'))
TARGETS = json.load(open('./data/DAVIS/target_seq.txt'))
SMILES=list(SMILES.values())
TARGETS=list(TARGETS.values())

In [5]:
all_9_folds={}
for i in [0,1,2]:
    for j in [0,1,2]:
        file_name = 'fold' +str(i) +str(j) 
        
        temp = open('./data/davis/DAVIS_9_FOLDS/' + file_name +'.pkl', 'rb')
        new_df = pd.read_pickle(temp)
        all_9_folds.update({file_name:new_df})
        temp.close()
        

In [6]:
def create_davis_test_train(test_fold_number,all_9_folds):
    test_protein_fold_id = test_fold_number[0]
    test_ligand_fold_id = test_fold_number[1]
    test_set = pd.DataFrame(columns = full_df.columns)
    train_set = pd.DataFrame(columns= full_df.columns)
    for i in [0,1,2]:
        for j in [0,1,2]:
            fold_name = 'fold' + str(i) + str(j)
            df = all_9_folds[fold_name]
            
            if str(i) == test_protein_fold_id and str(j) == test_ligand_fold_id:
                test_set = df.copy()
                
            if str(i) != test_protein_fold_id and str(j) != test_ligand_fold_id:
                train_set = pd.concat([train_set, df.copy()], ignore_index=True)
                
                
    return train_set, test_set


# Create train test split on these 9 folds
## fold_number is the id of fold. For example, test = fold00, train = fold 11,22,12,21

In [7]:
fold_number = '22'

In [8]:
train, test = create_davis_test_train(test_fold_number=fold_number, all_9_folds=all_9_folds)

# To ensure that there are no common targets or drugs in train and test


In [9]:
test_smiles = list(test['SMILES'])
test_targets = list(test['Target Sequence'])
train_smiles = list(train['SMILES'])
train_targets = list(train['Target Sequence'])

for i in test_smiles:
    if i in train_smiles:
        print("common entity present")
for i in test_targets:
    if i in train_targets:
        print("common entity present")


# Creating outer products for test set

In [10]:
outer_test_prods = []
for i,row in test.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = TARGETS.index(seq)
    smi_id = SMILES.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_test_prods.append([ki_x_kj])
outer_test_prods = np.array(outer_test_prods)
print(np.shape(outer_test_prods))

(3234, 1, 68, 442)


# creating outer products for train set

In [11]:
outer_train_prods = []
for i,row in train.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = TARGETS.index(seq)
    smi_id = SMILES.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_train_prods.append([ki_x_kj])
outer_train_prods = np.array(outer_train_prods)
print(np.shape(outer_train_prods))

(13570, 1, 68, 442)


In [12]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 20
# num_classes = 10
batch_size = 32
learning_rate = 0.001

In [13]:
class custom_dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, outer_prods, transform=None):
#         self.df = pd.read_csv(open(csv_file))
        self.df = dataframe
#         self.root_dir = root_dir
        self.transform = transform
        self.outer_prods = outer_prods
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        output = {'outer_product': self.outer_prods[idx] , 'Label':self.df.iloc[idx]['Label']}
        return output

In [14]:
train_dataset = custom_dataset(dataframe = train, outer_prods = outer_train_prods)
test_dataset = custom_dataset(dataframe = test, outer_prods = outer_test_prods)


In [15]:
train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader= torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [16]:
print(len(train_loader)*32, len(test_loader)*32)

13600 3264


In [17]:
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,32, 5).double()
        self.pool1 = nn.MaxPool2d(2,2).double()
        self.conv2 = nn.Conv2d(32,18,3).double()
        self.pool2 = nn.MaxPool2d(2,2).double()
        self.fc1 = nn.Linear(18*15*108, 128).double()
        self.fc2 = nn.Linear(128,1).double()
        self.dropout = nn.Dropout(0.1).double()
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,18*15*108)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        
        return x
    

In [18]:
# for i in test_loader:
#     a = i['outer_product']
#     b= i['Label']
#     break
# conv1 = nn.Conv2d(1,32,5).double()
# pool = nn.MaxPool2d(2,2).double()
# conv2 = nn.Conv2d(32,18,3).double()
# fc1 = nn.Linear(18*15*108, 128).double()
# fc2 = nn.Linear(128,1).double()
# dropout = nn.Dropout(0.1).double()
# x= conv1(a)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x= conv2(x)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x = x.view(-1,18*15*108)
# print(x.shape)
# x = dropout(x)
# print(x.shape)
# x = fc1(x)
# print(x.shape)
# x = fc2(x)
# print(x.shape)

In [19]:
model = ConvNet().to(device)

In [20]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Evaluation metrics

In [21]:
def rmse(y,f):
    rmse = math.sqrt(((y - f)**2).mean(axis=0))
    return rmse
def mse(y,f):
    mse = ((y - f)**2).mean(axis=0)
    return mse
def pearson(y,f):
    rp = np.corrcoef(y, f)[0,1]
    return rp
from lifelines.utils import concordance_index
def ci(y,f):
    return concordance_index(y,f)

In [22]:
def predicting(model, device, test_loader):
    model.eval()
    total_preds = np.array([])
    total_labels = np.array([])
    with torch.no_grad():
        for i in test_loader:
            images = i['outer_product']
            labels = i['Label']
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images) 
            outputs = outputs.cpu().detach().numpy().flatten()
            labels =labels.cpu().detach().numpy().flatten()
            total_preds = np.concatenate([total_preds, outputs])
            total_labels = np.concatenate([total_labels, labels])
    
    model.train()
    return total_labels, total_preds

# Train the model

In [23]:
model_file_name = './SAVED_MODELS/best_sim-CNN-DTA_davis_fold' + fold_number +  '.model'
result_file_name = './SAVED_MODELS/best_result_sim-CNNDTA_davis_fold'+fold_number + '.csv'

In [24]:
# Train the model
best_mse = 1000
best_ci = 0

total_step = len(train_loader)
for epoch in range(num_epochs):
    c=0
    for i in train_loader:
        c=c+1
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.flatten(), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, c, total_step, loss.item()))
    
    # taking best model so far
    G,P = predicting(model, device, test_loader)
    ret = [rmse(G, P), mse(G, P), pearson(G, P), ci(G, P)]
    if ret[1] < best_mse:
        torch.save(model.state_dict(), model_file_name)
        with open(result_file_name, 'w') as f:
            f.write(','.join(map(str, ret)))
        best_epoch = epoch+1
        best_mse = ret[1]
        best_ci = ret[-1]
        best_r = ret[2]
        
        print('rmse improved at epoch ', best_epoch,
                      '; best_mse,best_ci,best_r:', best_mse, best_ci,best_r)
        
        

/home2/kanakala.ganesh/miniconda3/envs/geometric/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/20], Step [1/425], Loss: 27.3477
Epoch [1/20], Step [2/425], Loss: 21.0491
Epoch [1/20], Step [3/425], Loss: 10.1751
Epoch [1/20], Step [4/425], Loss: 2.5527
Epoch [1/20], Step [5/425], Loss: 13.2555
Epoch [1/20], Step [6/425], Loss: 5.9211
Epoch [1/20], Step [7/425], Loss: 1.2694
Epoch [1/20], Step [8/425], Loss: 2.3780
Epoch [1/20], Step [9/425], Loss: 4.1737
Epoch [1/20], Step [10/425], Loss: 6.3530
Epoch [1/20], Step [11/425], Loss: 4.9736
Epoch [1/20], Step [12/425], Loss: 2.8855
Epoch [1/20], Step [13/425], Loss: 1.2901
Epoch [1/20], Step [14/425], Loss: 0.5864
Epoch [1/20], Step [15/425], Loss: 1.2549
Epoch [1/20], Step [16/425], Loss: 1.8425
Epoch [1/20], Step [17/425], Loss: 2.4740
Epoch [1/20], Step [18/425], Loss: 3.0940
Epoch [1/20], Step [19/425], Loss: 1.4206
Epoch [1/20], Step [20/425], Loss: 1.2485
Epoch [1/20], Step [21/425], Loss: 1.3771
Epoch [1/20], Step [22/425], Loss: 1.3510
Epoch [1/20], Step [23/425], Loss: 2.0990
Epoch [1/20], Step [24/425], Loss: 0.66

Epoch [1/20], Step [196/425], Loss: 0.6438
Epoch [1/20], Step [197/425], Loss: 0.9419
Epoch [1/20], Step [198/425], Loss: 0.9720
Epoch [1/20], Step [199/425], Loss: 1.1242
Epoch [1/20], Step [200/425], Loss: 1.1323
Epoch [1/20], Step [201/425], Loss: 0.8855
Epoch [1/20], Step [202/425], Loss: 1.2655
Epoch [1/20], Step [203/425], Loss: 0.3468
Epoch [1/20], Step [204/425], Loss: 0.7212
Epoch [1/20], Step [205/425], Loss: 0.9485
Epoch [1/20], Step [206/425], Loss: 0.4638
Epoch [1/20], Step [207/425], Loss: 0.8646
Epoch [1/20], Step [208/425], Loss: 0.8712
Epoch [1/20], Step [209/425], Loss: 0.9216
Epoch [1/20], Step [210/425], Loss: 1.0763
Epoch [1/20], Step [211/425], Loss: 0.6599
Epoch [1/20], Step [212/425], Loss: 1.0694
Epoch [1/20], Step [213/425], Loss: 1.5374
Epoch [1/20], Step [214/425], Loss: 0.4748
Epoch [1/20], Step [215/425], Loss: 0.4319
Epoch [1/20], Step [216/425], Loss: 0.4031
Epoch [1/20], Step [217/425], Loss: 1.2058
Epoch [1/20], Step [218/425], Loss: 0.3805
Epoch [1/20

Epoch [1/20], Step [387/425], Loss: 0.5410
Epoch [1/20], Step [388/425], Loss: 0.5479
Epoch [1/20], Step [389/425], Loss: 0.5855
Epoch [1/20], Step [390/425], Loss: 0.3088
Epoch [1/20], Step [391/425], Loss: 0.5009
Epoch [1/20], Step [392/425], Loss: 1.2253
Epoch [1/20], Step [393/425], Loss: 0.6495
Epoch [1/20], Step [394/425], Loss: 0.4327
Epoch [1/20], Step [395/425], Loss: 0.9072
Epoch [1/20], Step [396/425], Loss: 0.5810
Epoch [1/20], Step [397/425], Loss: 0.5044
Epoch [1/20], Step [398/425], Loss: 0.6669
Epoch [1/20], Step [399/425], Loss: 0.9032
Epoch [1/20], Step [400/425], Loss: 1.0752
Epoch [1/20], Step [401/425], Loss: 0.7833
Epoch [1/20], Step [402/425], Loss: 0.8741
Epoch [1/20], Step [403/425], Loss: 1.1547
Epoch [1/20], Step [404/425], Loss: 0.8024
Epoch [1/20], Step [405/425], Loss: 0.6336
Epoch [1/20], Step [406/425], Loss: 0.3376
Epoch [1/20], Step [407/425], Loss: 0.6751
Epoch [1/20], Step [408/425], Loss: 1.5186
Epoch [1/20], Step [409/425], Loss: 0.4442
Epoch [1/20

Epoch [2/20], Step [156/425], Loss: 0.6015
Epoch [2/20], Step [157/425], Loss: 0.2217
Epoch [2/20], Step [158/425], Loss: 0.4146
Epoch [2/20], Step [159/425], Loss: 0.2696
Epoch [2/20], Step [160/425], Loss: 0.8695
Epoch [2/20], Step [161/425], Loss: 0.4788
Epoch [2/20], Step [162/425], Loss: 0.5293
Epoch [2/20], Step [163/425], Loss: 0.3804
Epoch [2/20], Step [164/425], Loss: 0.7379
Epoch [2/20], Step [165/425], Loss: 0.1743
Epoch [2/20], Step [166/425], Loss: 0.8492
Epoch [2/20], Step [167/425], Loss: 0.5754
Epoch [2/20], Step [168/425], Loss: 0.2980
Epoch [2/20], Step [169/425], Loss: 0.9886
Epoch [2/20], Step [170/425], Loss: 0.6583
Epoch [2/20], Step [171/425], Loss: 0.6442
Epoch [2/20], Step [172/425], Loss: 0.5615
Epoch [2/20], Step [173/425], Loss: 0.6151
Epoch [2/20], Step [174/425], Loss: 0.4589
Epoch [2/20], Step [175/425], Loss: 0.2902
Epoch [2/20], Step [176/425], Loss: 0.7882
Epoch [2/20], Step [177/425], Loss: 0.7947
Epoch [2/20], Step [178/425], Loss: 0.2451
Epoch [2/20

...

Epoch [19/20], Step [16/425], Loss: 0.1188
Epoch [19/20], Step [17/425], Loss: 0.1260
Epoch [19/20], Step [18/425], Loss: 0.2410
Epoch [19/20], Step [19/425], Loss: 0.1628
Epoch [19/20], Step [20/425], Loss: 0.0906
Epoch [19/20], Step [21/425], Loss: 0.1381
Epoch [19/20], Step [22/425], Loss: 0.0633
Epoch [19/20], Step [23/425], Loss: 0.1847
Epoch [19/20], Step [24/425], Loss: 0.1296
Epoch [19/20], Step [25/425], Loss: 0.1015
Epoch [19/20], Step [26/425], Loss: 0.1343
Epoch [19/20], Step [27/425], Loss: 0.0887
Epoch [19/20], Step [28/425], Loss: 0.2765
Epoch [19/20], Step [29/425], Loss: 0.3834
Epoch [19/20], Step [30/425], Loss: 0.0730
Epoch [19/20], Step [31/425], Loss: 0.1745
Epoch [19/20], Step [32/425], Loss: 0.0674
Epoch [19/20], Step [33/425], Loss: 0.1201
Epoch [19/20], Step [34/425], Loss: 0.0726
Epoch [19/20], Step [35/425], Loss: 0.1250
Epoch [19/20], Step [36/425], Loss: 0.1006
Epoch [19/20], Step [37/425], Loss: 0.1725
Epoch [19/20], Step [38/425], Loss: 0.1533
Epoch [19/2

Epoch [19/20], Step [208/425], Loss: 0.1633
Epoch [19/20], Step [209/425], Loss: 0.0735
Epoch [19/20], Step [210/425], Loss: 0.0951
Epoch [19/20], Step [211/425], Loss: 0.0257
Epoch [19/20], Step [212/425], Loss: 0.0587
Epoch [19/20], Step [213/425], Loss: 0.0958
Epoch [19/20], Step [214/425], Loss: 0.1210
Epoch [19/20], Step [215/425], Loss: 0.1143
Epoch [19/20], Step [216/425], Loss: 0.1408
Epoch [19/20], Step [217/425], Loss: 0.0833
Epoch [19/20], Step [218/425], Loss: 0.2712
Epoch [19/20], Step [219/425], Loss: 0.2525
Epoch [19/20], Step [220/425], Loss: 0.1340
Epoch [19/20], Step [221/425], Loss: 0.0983
Epoch [19/20], Step [222/425], Loss: 0.0685
Epoch [19/20], Step [223/425], Loss: 0.3067
Epoch [19/20], Step [224/425], Loss: 0.0577
Epoch [19/20], Step [225/425], Loss: 0.0587
Epoch [19/20], Step [226/425], Loss: 0.2817
Epoch [19/20], Step [227/425], Loss: 0.2212
Epoch [19/20], Step [228/425], Loss: 0.1268
Epoch [19/20], Step [229/425], Loss: 0.1620
Epoch [19/20], Step [230/425], L

Epoch [19/20], Step [396/425], Loss: 0.1708
Epoch [19/20], Step [397/425], Loss: 0.1001
Epoch [19/20], Step [398/425], Loss: 0.2222
Epoch [19/20], Step [399/425], Loss: 0.0942
Epoch [19/20], Step [400/425], Loss: 0.0714
Epoch [19/20], Step [401/425], Loss: 0.1205
Epoch [19/20], Step [402/425], Loss: 0.1191
Epoch [19/20], Step [403/425], Loss: 0.0799
Epoch [19/20], Step [404/425], Loss: 0.1318
Epoch [19/20], Step [405/425], Loss: 0.0470
Epoch [19/20], Step [406/425], Loss: 0.2312
Epoch [19/20], Step [407/425], Loss: 0.0928
Epoch [19/20], Step [408/425], Loss: 0.2108
Epoch [19/20], Step [409/425], Loss: 0.1560
Epoch [19/20], Step [410/425], Loss: 0.1753
Epoch [19/20], Step [411/425], Loss: 0.1878
Epoch [19/20], Step [412/425], Loss: 0.2219
Epoch [19/20], Step [413/425], Loss: 0.0969
Epoch [19/20], Step [414/425], Loss: 0.2428
Epoch [19/20], Step [415/425], Loss: 0.2372
Epoch [19/20], Step [416/425], Loss: 0.1225
Epoch [19/20], Step [417/425], Loss: 0.0916
Epoch [19/20], Step [418/425], L

Epoch [20/20], Step [162/425], Loss: 0.1605
Epoch [20/20], Step [163/425], Loss: 0.0961
Epoch [20/20], Step [164/425], Loss: 0.0998
Epoch [20/20], Step [165/425], Loss: 0.3599
Epoch [20/20], Step [166/425], Loss: 0.1211
Epoch [20/20], Step [167/425], Loss: 0.0604
Epoch [20/20], Step [168/425], Loss: 0.0885
Epoch [20/20], Step [169/425], Loss: 0.1246
Epoch [20/20], Step [170/425], Loss: 0.0827
Epoch [20/20], Step [171/425], Loss: 0.0945
Epoch [20/20], Step [172/425], Loss: 0.0870
Epoch [20/20], Step [173/425], Loss: 0.0832
Epoch [20/20], Step [174/425], Loss: 0.0713
Epoch [20/20], Step [175/425], Loss: 0.1125
Epoch [20/20], Step [176/425], Loss: 0.0972
Epoch [20/20], Step [177/425], Loss: 0.1700
Epoch [20/20], Step [178/425], Loss: 0.2030
Epoch [20/20], Step [179/425], Loss: 0.0921
Epoch [20/20], Step [180/425], Loss: 0.1232
Epoch [20/20], Step [181/425], Loss: 0.0915
Epoch [20/20], Step [182/425], Loss: 0.0728
Epoch [20/20], Step [183/425], Loss: 0.1444
Epoch [20/20], Step [184/425], L

Epoch [20/20], Step [352/425], Loss: 0.1101
Epoch [20/20], Step [353/425], Loss: 0.0832
Epoch [20/20], Step [354/425], Loss: 0.0439
Epoch [20/20], Step [355/425], Loss: 0.1030
Epoch [20/20], Step [356/425], Loss: 0.2033
Epoch [20/20], Step [357/425], Loss: 0.3159
Epoch [20/20], Step [358/425], Loss: 0.1276
Epoch [20/20], Step [359/425], Loss: 0.1595
Epoch [20/20], Step [360/425], Loss: 0.1900
Epoch [20/20], Step [361/425], Loss: 0.2114
Epoch [20/20], Step [362/425], Loss: 0.1114
Epoch [20/20], Step [363/425], Loss: 0.1281
Epoch [20/20], Step [364/425], Loss: 0.0654
Epoch [20/20], Step [365/425], Loss: 0.1260
Epoch [20/20], Step [366/425], Loss: 0.1715
Epoch [20/20], Step [367/425], Loss: 0.2026
Epoch [20/20], Step [368/425], Loss: 0.4059
Epoch [20/20], Step [369/425], Loss: 0.1033
Epoch [20/20], Step [370/425], Loss: 0.1146
Epoch [20/20], Step [371/425], Loss: 0.1313
Epoch [20/20], Step [372/425], Loss: 0.0649
Epoch [20/20], Step [373/425], Loss: 0.1175
Epoch [20/20], Step [374/425], L

In [25]:
best_epoch

2

In [26]:
model.eval()
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
total_preds = np.array([])
total_labels = np.array([])
with torch.no_grad():
    correct = 0
    total = 0
    for i in test_loader:
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images) 
        outputs = outputs.cpu().detach().numpy().flatten()
        labels =labels.cpu().detach().numpy().flatten()
        total_preds = np.concatenate([total_preds, outputs])
        total_labels = np.concatenate([total_labels, labels])
#         total_preds = torch.cat(total_preds, outputs.cpu(), 0 )
#         total_labels = torch.cat(total_labels, labels.cpu(), 0)
#         break

In [27]:
G,P = total_labels, total_preds

In [28]:
rmse(G,P)

0.9883439113420596

In [29]:
mse(G,P)

0.9768236870869208

In [30]:
pearson(G,P)

0.18082188975402216

In [31]:
ci(G,P)

0.5640356401341357